In [1]:
import illustris_python as il
import numpy as np
from scipy.linalg import inv
from scipy.linalg import eig


#funciones de rotacion mediante momentum angular
def spherical_coords_from_vector(vector):
    x = vector[0] ; y = vector[1] ; z=vector[2]
    r = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arccos(z / r)
    if x > 0:
        phi = np.arctan(y/x)
    elif ((x < 0) and (y >= 0)):
        phi = np.arctan(y/x) + np.pi
    elif ((x < 0) and (y < 0)):
        phi = np.arctan(y/x) - np.pi
    elif ((x == 0) and (y > 0)):
        phi = np.pi / 2
    elif ((x == 0) and (y < 0)):
        phi = (-1 * (np.pi / 2))
    elif ((x == 0) and (y == 0)):
        phi = "INVALIDO"
    return( r, theta, phi)

def matrix_from_spherical( r, theta, phi):
    Rz = np.matrix([[np.cos(-phi),-np.sin(-phi),0],
                    [np.sin(-phi), np.cos(-phi),0],
                    [           0,            0,1]
                   ])
    
    Ry = np.matrix([[ np.cos(-theta),            0,np.sin(-theta)],
                    [            0,              1,             0],
                    [-np.sin(-theta),            0,np.cos(-theta)]
                   ])
    
    M = np.dot(Ry,Rz)
    return(M)

#funciones de rotacion mediante tensor de inercia
def inertia_tensor(Masas, Coordenadas, CentralPos  ):
    #calcula el tensor de inercia mediante las masas, coordenadas, y posicion central de un set de particulas
    #en caso de querer acotar por radio, formacion, etc, debe hacerse antes de correr el comando
    
    #reordenando el array
    xyz = np.squeeze(Coordenadas)
    if xyz.ndim == 1:
        xyz = np.reshape( xyz, (1,3) )
    
    #moviendo marco de referencia al de la posicion central
    for i in range(3):
        xyz[:,i] -= CentralPos[i]
        
    #Construccion del momento de inercia
    I = np.zeros( (3,3), dtype='float32' )

    I[0,0] = np.sum( Masas * (xyz[:,1]*xyz[:,1] + xyz[:,2]*xyz[:,2]) )
    I[1,1] = np.sum( Masas * (xyz[:,0]*xyz[:,0] + xyz[:,2]*xyz[:,2]) )
    I[2,2] = np.sum( Masas * (xyz[:,0]*xyz[:,0] + xyz[:,1]*xyz[:,1]) )
    I[0,1] = -1 * np.sum( Masas * (xyz[:,0]*xyz[:,1]) )
    I[0,2] = -1 * np.sum( Masas * (xyz[:,0]*xyz[:,2]) )
    I[1,2] = -1 * np.sum( Masas * (xyz[:,1]*xyz[:,2]) )
    I[1,0] = I[0,1]
    I[2,0] = I[0,2]
    I[2,1] = I[1,2]

    return(I)

def diagonalization_of_inertia(I):
    #entrega la matriz que diagonaliza el tensor de inercia
    autovalores,autovectores = eig(I)
    #reordenando autovectores y autovalores de manera ascendente
    ind_sort = np.argsort(autovalores)
    autovalores = autovalores[ind_sort]
    autovectores = autovectores[ind_sort]
    
    print("Autovalores \n",autovalores,"\n Autovectores \n",autovectores)
    #utiliza los autovectores para formar la matriz de rotacion
    rotation_matrix = inv(autovectores)
    
    return(rotation_matrix)

##Funciones de rotacion

def star_particles_rotated_once_eulermethod(subhaloID,snapNum,basepath):
    #ROTACION MEDIANTE METODO DE MOMENTO ANGULAR
    #constantes
    h = 0.6777
    
    #carga de datos
    stars = il.snapshot.loadSubhalo(basepath, snapNum, subhaloID, 'star', fields=['Masses','Coordinates','Velocities','GFM_Metallicity','Potential','GFM_StellarFormationTime'])
    subhalo = il.groupcat.loadSingle(basepath, snapNum, subhaloID=subhaloID)
    shPos = subhalo['SubhaloPos']
    rHalf = subhalo['SubhaloHalfmassRadType'][4]
    shVel = subhalo['SubhaloVel']
    
    #cambio unidades de las particulas a terminos fisicos
    stars['Masses'] *= (1e10/h)
    stars['Coordinates'] /= h
    stars['GFM_Metallicity'] /= 0.0127
    rHalf /= h
    shPos /= h
    print(rHalf)
    
    #distancias al centro
    vector_desde_centro = stars['Coordinates'] - shPos
    stars['Distance_to_center'] = np.array([np.linalg.norm(v) for v in vector_desde_centro])
    
    #filtrando por distancia al centro y por metalicidad similar a la solar
    wStars = np.where( (stars['GFM_Metallicity'] >= 1) & (stars['GFM_Metallicity'] <= 1.5) & (stars['Distance_to_center'] <= 2.0*rHalf) & (stars['GFM_StellarFormationTime'] > 0) )
    stars['Masses'] = stars['Masses'][wStars]
    stars['Coordinates'] = stars['Coordinates'][wStars]
    stars['GFM_Metallicity'] = stars['GFM_Metallicity'][wStars]
    stars['Distance_to_center'] = stars['Distance_to_center'][wStars]
    stars['Velocities'] = stars['Velocities'][wStars]
    stars['Potential'] = stars['Potential'][wStars]
    stars['count'] = len(stars['Masses'])
    
    #cambio de marco de referencia
    stars['Coordinates'] -= shPos
    stars['Velocities'] -= shVel
    
    
    #calculo del momento angular
    stars['Angular_Momentum'] = np.cross(stars['Coordinates'],stars['Velocities'])
    total_angular_momentum = sum(stars['Angular_Momentum'])
    
    r, theta, phi = spherical_coords_from_vector(total_angular_momentum)
    M = matrix_from_spherical(r, theta, phi)
    print(np.dot(M, total_angular_momentum))
    
    
    stars['Coordinates'] = np.array([np.array(np.dot(M, Coord))[0] for Coord in stars['Coordinates']])
    stars['Velocities'] = np.array([np.array(np.dot(M, Velocidad))[0] for Velocidad in stars['Velocities']])
    
    return(stars, M)

def star_particles_rotated_once(subhaloID,snapNum,basepath):
    #ROTACION MEDIANTE EL TENSOR DE INERCIA
    #constantes
    h = 0.6777
    
    #carga de datos
    stars = il.snapshot.loadSubhalo(basepath, snapNum, subhaloID, 'star', fields=['Masses','Coordinates','Velocities','GFM_Metallicity','Potential','GFM_StellarFormationTime'])
    subhalo = il.groupcat.loadSingle(basepath, snapNum, subhaloID=subhaloID)
    shPos = subhalo['SubhaloPos']
    rHalf = subhalo['SubhaloHalfmassRadType'][4]
    shVel = subhalo['SubhaloVel']
    
    #cambio unidades de las particulas a terminos fisicos
    stars['Masses'] *= (1e10/h)
    stars['Coordinates'] /= h
    stars['GFM_Metallicity'] /= 0.0127
    rHalf /= h
    shPos /= h
    print(rHalf)
    
    #distancias al centro
    vector_desde_centro = stars['Coordinates'] - shPos
    stars['Distance_to_center'] = np.array([np.linalg.norm(v) for v in vector_desde_centro])
    
    #filtrando por distancia al centro y por metalicidad similar a la solar
    wStars = np.where( (stars['GFM_Metallicity'] >= 1) & (stars['GFM_Metallicity'] <= 1.5) & (stars['Distance_to_center'] <= 2.0*rHalf) & (stars['GFM_StellarFormationTime'] > 0) )
    stars['Masses'] = stars['Masses'][wStars]
    stars['Coordinates'] = stars['Coordinates'][wStars]
    stars['GFM_Metallicity'] = stars['GFM_Metallicity'][wStars]
    stars['Distance_to_center'] = stars['Distance_to_center'][wStars]
    stars['Velocities'] = stars['Velocities'][wStars]
    stars['Potential'] = stars['Potential'][wStars]
    stars['count'] = len(stars['Masses'])
    
    
    #obtencion del tensor de inercia
    I = np.matrix(inertia_tensor(stars['Masses'], stars['Coordinates'], shPos ))
    #obtencion de la matriz de rotacion
    S = np.matrix(diagonalization_of_inertia(I))
    S_1 = np.matrix(inv(S))
    print(S_1 * I * S,"|<-----Tensor de inercia Diagonalizado")
    stars['Coordinates'] = np.dot(stars['Coordinates'], S)
    stars['Velocities'] -= shVel
    stars['Velocities'] = np.dot(stars['Velocities'], S)
    
    return(stars, S, S_1)


## Funciones de calculo de circularidades individuales, las deja registradas en el diccionario de las particulas

def circularities_eulermethod(subhaloID,snapNum,basepath,radius_fraction,metallicity_cutoff):
    #calcula las circularidades individuales dentro de un radio, y por sobre un valor de metalicidad, rotando con el metodo de momento angular y angulos
    h = 0.677 #constante a-dimensional de hubble
    stars, M = star_particles_rotated_once_eulermethod(494709,99,'sims.TNG/TNG50-1/output')
    
    
    #rotando todas las particulas
    stars = il.snapshot.loadSubhalo(basepath,snapNum,subhaloID, 'star', fields=['Masses','Coordinates','Velocities','GFM_Metallicity','Potential','GFM_StellarFormationTime'])

    subhalo = il.groupcat.loadSingle(basepath, snapNum, subhaloID=subhaloID)
    shPos = subhalo['SubhaloPos']
    shVel = subhalo['SubhaloVel']
    rHalf = subhalo['SubhaloHalfmassRadType'][4]


    stars['Coordinates'] -= shPos
    stars['Velocities']  -= shVel
    
    #pasando datos a valores fisicos
    stars['Masses'] *= (1e10/h)
    stars['Coordinates'] /= h
    stars['GFM_Metallicity'] /= 0.0127
    rHalf /= h
    shPos /= h
    
    #distancia al centro
    stars['Distance_to_center'] = np.array([np.linalg.norm(v) for v in stars['Coordinates']])
    
    #filtrando por radio y metalicidad
    if metallicity_cutoff != False:
        wStars = np.where((stars['Distance_to_center'] <= radius_fraction*rHalf) & (stars['GFM_Metallicity'] >= metallicity_cutoff) & (stars['GFM_StellarFormationTime'] > 0))
    else:
        wStars = np.where((stars['Distance_to_center'] <= radius_fraction*rHalf) & (stars['GFM_StellarFormationTime'] > 0))
        
    stars['Masses'] = stars['Masses'][wStars]
    stars['Velocities'] = stars['Velocities'][wStars]
    stars['Coordinates'] = stars['Coordinates'][wStars]
    stars['GFM_Metallicity'] = stars['GFM_Metallicity'][wStars]
    stars['Potential'] = stars['Potential'][wStars]
    stars['GFM_StellarFormationTime'] = stars['GFM_StellarFormationTime'][wStars]
    
    
    #filtrado
    
    
    

    stars['Coordinates'] = np.array([np.array(np.dot(M, coords))[0] for coords in stars['Coordinates']])
    stars['Velocities'] = np.array([np.array(np.dot(M, vels))[0] for vels in stars['Coordinates']])


    stars['Angular_Momentum'] = np.cross(stars['Coordinates'],stars['Velocities'])
    print("-----------------------\n",sum(stars['Angular_Momentum']))

    norm_velocities = [np.linalg.norm(v) for v in stars['Velocities']]
    norm_velocities = np.array(norm_velocities)
    norm_velocities

    stars['Specific_Energy'] = (0.5*norm_velocities**2) + stars['Potential']

    indices_ordenados = np.argsort(stars['Specific_Energy'])

    stars['Masses'] = stars['Masses'][indices_ordenados]
    stars['Coordinates'] = stars['Coordinates'][indices_ordenados]
    stars['Velocities'] = stars['Velocities'][indices_ordenados]
    stars['GFM_Metallicity'] = stars['GFM_Metallicity'][indices_ordenados]
    stars['Potential'] = stars['Potential'][indices_ordenados]
    stars['GFM_StellarFormationTime'] = stars['GFM_StellarFormationTime'][indices_ordenados]
    #stars['Distance_to_center'] = stars['Distance_to_center'][indices_ordenados]
    stars['Angular_Momentum'] = stars['Angular_Momentum'][indices_ordenados]
    stars['Specific_Energy'] = stars['Specific_Energy'][indices_ordenados]

    jz = stars['Angular_Momentum'][:,2]
    npm = 50
    max_jz = [np.max(jz[:i+npm]) if i < npm else np.max(jz[i-npm:]) if i > len(jz)-npm else np.max(jz[i-npm:i+npm]) for i in range(len(jz))]
    max_jz = np.array(max_jz)


    circularity = jz/max_jz
    stars['Circularity'] = circularity

    return(stars)

def circularities_diagmethod(subhaloID,snapNum,basepath,radius_fraction,metallicity_cutoff):
    #calcula las circularidades individuales, mediante el metodo de rotacion por diagonalizacion del tensor de inercia
    h = 0.677 #constante a-dimensional de hubble
    stars, S, S_1 = star_particles_rotated_once(494709,99,'sims.TNG/TNG50-1/output')
    
    
    #rotando todas las particulas
    stars = il.snapshot.loadSubhalo(basepath,snapNum,subhaloID, 'star', fields=['Masses','Coordinates','Velocities','GFM_Metallicity','Potential','GFM_StellarFormationTime'])

    subhalo = il.groupcat.loadSingle(basepath, snapNum, subhaloID=subhaloID)
    shPos = subhalo['SubhaloPos']
    shVel = subhalo['SubhaloVel']
    rHalf = subhalo['SubhaloHalfmassRadType'][4]


    stars['Coordinates'] -= shPos
    stars['Velocities']  -= shVel
    
    #pasando datos a valores fisicos
    stars['Masses'] *= (1e10/h)
    stars['Coordinates'] /= h
    stars['GFM_Metallicity'] /= 0.0127
    rHalf /= h
    shPos /= h
    
    #distancia al centro
    stars['Distance_to_center'] = np.array([np.linalg.norm(v) for v in stars['Coordinates']])
    
    #filtrando por radio y metalicidad
    if metallicity_cutoff != False:
        wStars = np.where((stars['Distance_to_center'] <= radius_fraction*rHalf) & (stars['GFM_Metallicity'] >= metallicity_cutoff) & (stars['GFM_StellarFormationTime'] > 0))
    else:
        wStars = np.where( (stars['Distance_to_center'] <= radius_fraction*rHalf) & (stars['GFM_StellarFormationTime'] > 0))
        
    stars['Masses'] = stars['Masses'][wStars]
    stars['Velocities'] = stars['Velocities'][wStars]
    stars['Coordinates'] = stars['Coordinates'][wStars]
    stars['GFM_Metallicity'] = stars['GFM_Metallicity'][wStars]
    stars['Potential'] = stars['Potential'][wStars]
    
    
    #filtrado

    stars['Coordinates'] = np.array([np.array(np.dot(coords,S))[0] for coords in stars['Coordinates']])
    stars['Velocities'] = np.array([np.array(np.dot(vels,S))[0] for vels in stars['Coordinates']])


    stars['Angular_Momentum'] = np.cross(stars['Coordinates'],stars['Velocities'])
    print("-----------------------\n",sum(stars['Angular_Momentum']))

    norm_velocities = [np.linalg.norm(v) for v in stars['Velocities']]
    norm_velocities = np.array(norm_velocities)
    norm_velocities

    stars['Specific_Energy'] = (0.5*norm_velocities**2) + stars['Potential']

    indices_ordenados = np.argsort(stars['Specific_Energy'])

    stars['Masses'] = stars['Masses'][indices_ordenados]
    stars['Coordinates'] = stars['Coordinates'][indices_ordenados]
    stars['Velocities'] = stars['Velocities'][indices_ordenados]
    stars['GFM_Metallicity'] = stars['GFM_Metallicity'][indices_ordenados]
    stars['Potential'] = stars['Potential'][indices_ordenados]
    #stars['Distance_to_center'] = stars['Distance_to_center'][indices_ordenados]
    stars['Angular_Momentum'] = stars['Angular_Momentum'][indices_ordenados]
    stars['Specific_Energy'] = stars['Specific_Energy'][indices_ordenados]

    jz = stars['Angular_Momentum'][:,2]
    npm = 50
    max_jz = [np.max(jz[:i+npm]) if i < npm else np.max(jz[i-npm:]) if i > len(jz)-npm else np.max(jz[i-npm:i+npm]) for i in range(len(jz))]
    max_jz = np.array(max_jz)


    circularity = jz/max_jz
    stars['Circularity'] = circularity

    return(stars)

# Test con particulas perfectas

In [2]:
Coordenadas = np.array([[11,11,11],
                        [12,10,11],
                        [10,12,11],
                        [12,12,13],
                        [10,10,9]
                       ])

Velocidades = np.array([[0,0,0],
                        [1,1,2],
                        [-1,-1,-2],
                        [-2,2,0],
                        [2,-2,0]
                       ])

Masas = [1,1,1,1,1]

CentralPos = [11,11,11]

Coordenadas -= CentralPos



Momentos_Angulares = np.cross(Coordenadas,Velocidades)
total_angular_momentum = sum(Momentos_Angulares)
r, theta, phi = spherical_coords_from_vector(total_angular_momentum)
M = matrix_from_spherical(r, theta, phi)





# nuevas_coords = np.dot(Coordenadas,M)
# nuevas_vels = np.dot(Velocidades,M)

nuevas_coords = np.array([np.array(np.dot(M, Coord))[0] for Coord in Coordenadas])
nuevas_vels = np.array([np.array(np.dot(M, Velocidad))[0] for Velocidad in Velocidades])

nuevas_coords = np.round(nuevas_coords,2)
nuevas_vels = np.round(nuevas_vels,2)

print(np.dot(M, total_angular_momentum))
print("Coordenadas\n",nuevas_coords,"\n","Velocidades\n",nuevas_vels)
print(sum(np.cross(nuevas_coords,nuevas_vels)),"\n",np.cross(nuevas_coords,nuevas_vels))

[[ -8.88178420e-16  -1.33226763e-15   2.07846097e+01]]
Coordenadas
 [[ 0.    0.    0.  ]
 [ 0.    1.41  0.  ]
 [-0.   -1.41 -0.  ]
 [-2.45  0.    0.  ]
 [ 2.45 -0.    0.  ]] 
 Velocidades
 [[ 0.    0.    0.  ]
 [-2.45  0.    0.  ]
 [ 2.45 -0.    0.  ]
 [-0.   -2.83 -0.  ]
 [ 0.    2.83  0.  ]]
[  0.      0.     20.776] 
 [[ 0.      0.      0.    ]
 [ 0.     -0.      3.4545]
 [-0.      0.      3.4545]
 [ 0.     -0.      6.9335]
 [-0.      0.      6.9335]]


In [9]:
Coordenadas = np.array([[11,11,11],
                        [12,10,11],
                        [10,12,11],
                        [12,12,13],
                        [10,10,9]
                       ])

Velocidades = np.array([[0,0,0],
                        [1,1,2],
                        [-1,-1,-2],
                        [-2,2,0],
                        [2,-2,0]
                       ])

Masas = [1,1,1,1,1]

CentralPos = [11,11,11]

I = np.matrix(inertia_tensor(Masas, Coordenadas, CentralPos  ))
S = np.matrix(diagonalization_of_inertia(I))
S_1 = np.matrix(inv(S))

nuevas_coords = np.dot(Coordenadas, S_1)
nuevas_vels = np.dot(Velocidades, S_1)

print(S * I * S_1)
nuevas_coords = np.round(nuevas_coords,2)
nuevas_vels = np.round(nuevas_vels,2)
print("Coordenadas\n",nuevas_coords,"\n","Velocidades\n",nuevas_vels)
print(sum(np.cross(nuevas_coords,nuevas_vels)),"\n",np.cross(nuevas_coords,nuevas_vels))

Autovalores 
 [  3.99999857+0.j  11.99999905+0.j  16.00000000+0.j] 
 Autovectores 
 [[  4.08248186e-01  -7.07106650e-01   5.77350080e-01]
 [  4.08248365e-01   7.07106829e-01   5.77350378e-01]
 [  8.16496611e-01   5.28359259e-08  -5.77350318e-01]]
[[  3.99999976e+00  -1.14330862e-06   2.47990044e-07]
 [  1.36271956e-06   1.19999990e+01  -7.16760340e-08]
 [  3.67589109e-07   5.98126917e-07   1.60000000e+01]]
Coordenadas
 [[ 0.    0.    0.  ]
 [-0.   -1.41 -0.  ]
 [ 0.    1.41  0.  ]
 [ 2.45  0.    0.  ]
 [-2.45 -0.    0.  ]] 
 Velocidades
 [[ 0.    0.    0.  ]
 [ 2.45  0.    0.  ]
 [-2.45 -0.    0.  ]
 [ 0.    2.83  0.  ]
 [-0.   -2.83 -0.  ]]
[  0.      0.     20.776] 
 [[ 0.      0.      0.    ]
 [ 0.      0.      3.4545]
 [ 0.     -0.      3.4545]
 [ 0.      0.      6.9335]
 [ 0.     -0.      6.9335]]


matrix([[  4.08248186e-01,  -7.07106590e-01,   5.77350080e-01],
        [  4.08248365e-01,   7.07106829e-01,   5.77350438e-01],
        [  8.16496611e-01,   4.75679798e-08,  -5.77350259e-01]], dtype=float32)

# Rotation of particles

In [2]:
stars, M = star_particles_rotated_once_eulermethod(394621,99,'sims.TNG/TNG50-1/output')

stars['Angular_Momentum'] = np.cross(stars['Coordinates'],stars['Velocities'])
print("-----------------------\n",sum(stars['Angular_Momentum']))

np.savetxt('datosPOS_estrellas_eulermethod.csv',stars['Coordinates'],delimiter=',')
np.savetxt('datosVEL_estrellas_eulermethod.csv',stars['Velocities'],delimiter=',')

6.31126943056
[[ -2.10348428e-09   4.71124550e-09   1.46322505e+08]]
-----------------------
 [ -2.61551008e-07  -7.69326363e-07   1.46322505e+08]


In [ ]:
# Rotando las coordenadas usando M

subhaloID = 394621; snapNum = 99; basepath = 'sims.TNG/TNG50-1/output'

stars = il.snapshot.loadSubhalo(basepath,snapNum,subhaloID, 'star', fields=['Masses','Coordinates','Velocities','GFM_Metallicity','Potential'])

subhalo = il.groupcat.loadSingle(basepath, snapNum, subhaloID=subhaloID)
shPos = subhalo['SubhaloPos']
shVel = subhalo['SubhaloVel']


stars['Coordinates'] -= shPos
stars['Velocities']  -= shVel

stars['Coordinates'] = np.array([np.array(np.dot(M, coords))[0] for coords in stars['Coordinates']])
stars['Velocities'] = np.array([np.array(np.dot(M, vels))[0] for vels in stars['Coordinates']])

np.savetxt('datosPOS_ALLestrellas_eulermethod.csv',stars['Coordinates'],delimiter=',')

In [22]:
#stars = il.snapshot.loadSubhalo('sims.TNG/TNG50-1/output',99,494709, 'star', fields=['Masses','Coordinates','GFM_Metallicity'])
stars['GFM_Metallicity'] /= 0.0127
wStars = np.where( (stars['GFM_Metallicity'] >= 1) )
stars['Masses'] = stars['Masses'][wStars]
stars['Coordinates'] = stars['Coordinates'][wStars]
stars['Velocities'] = stars['Velocities'][wStars]
stars['GFM_Metallicity'] = stars['GFM_Metallicity'][wStars]
stars['Potential'] = stars['Potential'][wStars]


np.savetxt('datosPOS_RAW_estrellasMETALICAS.csv',stars['Coordinates'],delimiter=',')

### ------------------------Zona del tensor de inercia---------------------------------

In [16]:
stars, S, S_1 = star_particles_rotated_once(394621,99,'sims.TNG/TNG50-1/output')

stars['Angular_Momentum'] = np.cross(stars['Coordinates'],stars['Velocities'])
print("-----------------------\n",sum(stars['Angular_Momentum']))

np.savetxt('datosPOS_estrellas.csv',stars['Coordinates'],delimiter=',')
np.savetxt('datosVEL_estrellas.csv',stars['Velocities'],delimiter=',')

8.78085600375
Autovalores 
 [  5.25069582e+11+0.j   5.30789761e+11+0.j   9.86400227e+11+0.j] 
 Autovectores 
 [[-0.61993992 -0.38497245  0.68372154]
 [ 0.32870445  0.66380388  0.6717962 ]
 [ 0.71248007 -0.64121807  0.28498179]]
[[  5.25288964e+11   5.46697088e+08   9.99753523e+09]
 [  5.46606592e+08   5.31649364e+11   1.97927813e+10]
 [  9.99541043e+09   1.97921014e+10   9.85320718e+11]] |<-----Tensor de inercia Diagonalizado
-----------------------
 [ -6.45364045e+06  -1.21415655e+07  -2.83065699e+08]


In [21]:
#Rotando las coordenadas usando S

subhaloID = 394621; snapNum = 99; basepath = 'sims.TNG/TNG50-1/output'

stars = il.snapshot.loadSubhalo(basepath,snapNum,subhaloID, 'star', fields=['Masses','Coordinates','Velocities','GFM_Metallicity','Potential'])

subhalo = il.groupcat.loadSingle(basepath, snapNum, subhaloID=subhaloID)
shPos = subhalo['SubhaloPos']
shVel = subhalo['SubhaloVel']


stars['Coordinates'] -= shPos
stars['Velocities']  -= shVel

stars['Coordinates'] = np.array([np.array(np.dot(coords,S))[0] for coords in stars['Coordinates']])
stars['Velocities'] = np.array([np.array(np.dot(vels,S))[0] for vels in stars['Coordinates']])

np.savetxt('datosPOS_ALLestrellas.csv',stars['Coordinates'],delimiter=',')

In [8]:
#[ for coords in stars['Coordinates']]

stars['Coordinates']

array([[  6.45571035e-02,   1.60487193e-02,   3.26976615e-02],
       [  5.47892934e-02,   1.08875240e-02,   4.02730159e-02],
       [  5.76683883e-02,  -1.46554201e-02,   2.07103822e-04],
       ..., 
       [ -1.24447227e+02,   1.29459931e+02,   4.31013478e+01],
       [ -7.77739622e+01,   1.85929651e+02,   7.84110401e+01],
       [ -1.25447041e+02,   1.29471303e+02,   4.45977648e+01]])

In [13]:
stars = il.snapshot.loadSubhalo('sims.TNG/TNG50-1/output',99,394621, 'star', fields=['Masses','Coordinates','Velocities','GFM_Metallicity','Potential','GFM_StellarFormationTime'])
stars['GFM_Metallicity'] /= 0.0127
wStars = np.where( (stars['GFM_Metallicity'] >= 1) & (stars['GFM_Metallicity'] <= 1.5) & (stars['GFM_StellarFormationTime'] > 0) )
stars['Masses'] = stars['Masses'][wStars]
stars['Coordinates'] = stars['Coordinates'][wStars]
stars['Velocities'] = stars['Velocities'][wStars]
stars['GFM_Metallicity'] = stars['GFM_Metallicity'][wStars]
stars['Potential'] = stars['Potential'][wStars]
stars['count'] = len(stars['Masses'])


stars

{'count': 282204,
 'Masses': array([  2.51052711e-06,   3.70294310e-06,   2.83330860e-06, ...,
          3.66798145e-06,   3.92988295e-06,   3.74699653e-06], dtype=float32),
 'Coordinates': array([[ 15170.19751   ,   1034.44337252,  20953.56218376],
        [ 15170.32788323,   1034.48864695,  20953.54129038],
        [ 15170.2878562 ,   1034.56174559,  20953.57123232],
        ..., 
        [ 15199.16076493,   1071.68052335,  20962.25935723],
        [ 15179.30979124,   1049.71820925,  20931.01298485],
        [ 15154.20796611,   1015.6344634 ,  20920.62725251]]),
 'Velocities': array([[  78.54011536,  -96.44454193,    3.4515326 ],
        [  34.4197998 ,  -38.3124733 ,  -71.94756317],
        [  28.24641991, -128.80912781,   15.34276295],
        ..., 
        [ 162.14784241,   45.34522247,  189.53048706],
        [ 274.83538818,  154.23406982,  131.4773407 ],
        [ 231.75782776,  165.02409363,  -43.55035019]], dtype=float32),
 'GFM_Metallicity': array([ 1.3435663 ,  1.08913732,  

# Calculation of circularity

In [3]:
stars, M = star_particles_rotated_once_eulermethod(394621,99,'sims.TNG/TNG50-1/output')

stars['Angular_Momentum'] = np.cross(stars['Coordinates'],stars['Velocities'])
print("-----------------------\n",sum(stars['Angular_Momentum']))

norm_velocities = [np.linalg.norm(v) for v in stars['Velocities']]
norm_velocities = np.array(norm_velocities)
norm_velocities

stars['Specific_Energy'] = (0.5*norm_velocities**2) + stars['Potential']

indices_ordenados = np.argsort(stars['Specific_Energy'])

stars['Masses'] = stars['Masses'][indices_ordenados]
stars['Coordinates'] = stars['Coordinates'][indices_ordenados]
stars['Velocities'] = stars['Velocities'][indices_ordenados]
stars['GFM_Metallicity'] = stars['GFM_Metallicity'][indices_ordenados]
stars['Potential'] = stars['Potential'][indices_ordenados]
#stars['Distance_to_center'] = stars['Distance_to_center'][indices_ordenados]
stars['Angular_Momentum'] = stars['Angular_Momentum'][indices_ordenados]
stars['Specific_Energy'] = stars['Specific_Energy'][indices_ordenados]

jz = stars['Angular_Momentum'][:,2]
npm = 50
max_jz = [np.max(jz[:i+npm]) if i < npm else np.max(jz[i-npm:]) if i > len(jz)-npm else np.max(jz[i-npm:i+npm]) for i in range(len(jz))]
max_jz = np.array(max_jz)


circularity = jz/max_jz
stars['Circularity'] = circularity

np.savetxt('datos_circularidades_eulermethod.csv',stars['Circularity'],delimiter=',')

6.31126943056
[[ -2.10348428e-09   4.71124550e-09   1.46322505e+08]]
-----------------------
 [ -2.61551008e-07  -7.69326363e-07   1.46322505e+08]


In [15]:
stars, M = star_particles_rotated_once_eulermethod(394621,99,'sims.TNG/TNG50-1/output')

#rotando todas las particulas
subhaloID = 394621; snapNum = 99; basepath = 'sims.TNG/TNG50-1/output'

stars = il.snapshot.loadSubhalo(basepath,snapNum,subhaloID, 'star', fields=['Masses','Coordinates','Velocities','GFM_Metallicity','Potential'])

subhalo = il.groupcat.loadSingle(basepath, snapNum, subhaloID=subhaloID)
shPos = subhalo['SubhaloPos']
shVel = subhalo['SubhaloVel']


stars['Coordinates'] -= shPos
stars['Velocities']  -= shVel

stars['Coordinates'] = np.array([np.array(np.dot(M, coords))[0] for coords in stars['Coordinates']])
stars['Velocities'] = np.array([np.array(np.dot(M, vels))[0] for vels in stars['Coordinates']])

stars['Angular_Momentum'] = np.cross(stars['Coordinates'],stars['Velocities'])
print("-----------------------\n",sum(stars['Angular_Momentum']))

norm_velocities = [np.linalg.norm(v) for v in stars['Velocities']]
norm_velocities = np.array(norm_velocities)
norm_velocities

stars['Specific_Energy'] = (0.5*norm_velocities**2) + stars['Potential']

indices_ordenados = np.argsort(stars['Specific_Energy'])

stars['Masses'] = stars['Masses'][indices_ordenados]
stars['Coordinates'] = stars['Coordinates'][indices_ordenados]
stars['Velocities'] = stars['Velocities'][indices_ordenados]
stars['GFM_Metallicity'] = stars['GFM_Metallicity'][indices_ordenados]
stars['Potential'] = stars['Potential'][indices_ordenados]
#stars['Distance_to_center'] = stars['Distance_to_center'][indices_ordenados]
stars['Angular_Momentum'] = stars['Angular_Momentum'][indices_ordenados]
stars['Specific_Energy'] = stars['Specific_Energy'][indices_ordenados]

jz = stars['Angular_Momentum'][:,2]
npm = 50
max_jz = [np.max(jz[:i+npm]) if i < npm else np.max(jz[i-npm:]) if i > len(jz)-npm else np.max(jz[i-npm:i+npm]) for i in range(len(jz))]
max_jz = np.array(max_jz)


circularity = jz/max_jz
stars['Circularity'] = circularity

np.savetxt('datosALL_circularidades_eulermethod.csv',stars['Circularity'],delimiter=',')

8.78085600375
[[ -7.53456484e-09   2.66504908e-08   2.83398982e+08]]
-----------------------
 [-44373291.72880507 -63252030.73685679  62450902.51319665]


### Zona del tensor de inercia

In [26]:
#stars, S, S_1 = star_particles_rotated_once(494709,99,'sims.TNG/TNG50-1/output')

stars['Angular_Momentum'] = np.cross(stars['Coordinates'],stars['Velocities'])
print("-----------------------\n",sum(stars['Angular_Momentum']))

norm_velocities = [np.linalg.norm(v) for v in stars['Velocities']]
norm_velocities = np.array(norm_velocities)
norm_velocities

stars['Specific_Energy'] = (0.5*norm_velocities**2) + stars['Potential']

indices_ordenados = np.argsort(stars['Specific_Energy'])

stars['Masses'] = stars['Masses'][indices_ordenados]
stars['Coordinates'] = stars['Coordinates'][indices_ordenados]
stars['Velocities'] = stars['Velocities'][indices_ordenados]
stars['GFM_Metallicity'] = stars['GFM_Metallicity'][indices_ordenados]
stars['Potential'] = stars['Potential'][indices_ordenados]
#stars['Distance_to_center'] = stars['Distance_to_center'][indices_ordenados]
stars['Angular_Momentum'] = stars['Angular_Momentum'][indices_ordenados]
stars['Specific_Energy'] = stars['Specific_Energy'][indices_ordenados]

jz = stars['Angular_Momentum'][:,2]
npm = 50
max_jz = [np.max(jz[:i+npm]) if i < npm else np.max(jz[i-npm:]) if i > len(jz)-npm else np.max(jz[i-npm:i+npm]) for i in range(len(jz))]
max_jz = np.array(max_jz)


circularity = jz/max_jz
stars['Circularity'] = circularity

-----------------------
 [-19244146.5566427  -20255806.00022464  23447518.29735858]


In [26]:
jz[:2+40]

array([  2.10025988e-05,  -7.67667471e-04,  -1.51160688e-03,
        -4.94670768e-04,  -5.83025049e-04,  -4.29936813e-05,
         4.29105818e-04,  -4.92019900e-04,  -2.21895137e-04,
        -7.88078660e-04,  -1.07494267e-04,  -6.77764590e-04,
        -5.96346582e-06,  -4.22422577e-04,  -2.46654411e-04,
        -3.15431879e-05,   7.19090493e-06,  -6.84717240e-04,
         1.55077516e-04,  -1.76478565e-04,  -2.74185009e-04,
        -1.91988891e-04,  -1.15156182e-04,  -4.13292903e-03,
         2.60557693e-05,  -1.12504778e-04,   1.74699917e-04,
        -3.71659328e-04,  -6.69931845e-04,   1.24437116e-04,
         8.58128268e-04,   4.35300312e-04,  -2.57825943e-03,
        -8.23083619e-04,  -3.61827448e-04,   1.16789438e-04,
        -2.24087680e-03,  -3.50379662e-05,  -1.18599941e-05,
        -7.06528025e-04,  -3.30796004e-04,  -2.37942352e-07])

In [29]:
np.max(abs(jz[:2+40])),jz[2]

(0.00413292902635782, -0.0015116068778327895)

In [27]:
stars['Circularity']

array([ 0.11548516,  0.33061163,  0.24555513, ...,  0.93866259,
        1.        ,  0.80196756])

In [28]:
np.savetxt('datosALL_circularidades.csv',stars['Circularity'],delimiter=',')

In [8]:
indices_ordenados

array([    144,      40,      37, ..., 1317302, 1317314, 1317354])

In [35]:
## Utilizando fu nciones creadas para la circularidad
subhaloID = 394621
snapNum = 99
basepath = 'sims.TNG/TNG50-1/output'
radius_fraction = 5.0
metallicity_cutoff = False


stars = circularities_eulermethod(subhaloID,snapNum,basepath,radius_fraction,metallicity_cutoff)
stars['count'] = len(stars['Masses'])

8.78085600375
[[ -7.53456484e-09   2.66504908e-08   2.83398982e+08]]
-----------------------
 [ -7.12300588e+07  -9.62122108e+07   1.07225492e+08]


In [36]:
stars

{'count': 1301958,
 'Masses': array([ 46668.125     ,  44963.171875  ,  49238.66796875, ...,
         61256.86328125,  88369.1171875 ,  46834.1640625 ], dtype=float32),
 'Coordinates': array([[  3.70749442e-02,   1.80852144e-02,  -7.44632376e-03],
        [  6.85675418e-02,   2.32580388e-02,   1.58983190e-02],
        [  5.44102610e-02,  -2.34510366e-02,  -6.30223763e-03],
        ..., 
        [ -1.68256799e+01,  -8.62648554e+00,  -3.89990453e+01],
        [ -1.82386436e+01,  -1.74531412e+01,  -3.56744146e+01],
        [ -1.28218567e+01,  -5.67890127e+00,  -4.15845637e+01]]),
 'Velocities': array([[  1.23118517e-02,   3.80006862e-02,   1.27059183e-02],
        [ -3.04452764e-03,   6.26014197e-02,   3.95852727e-02],
        [  2.45123660e-02,   1.80967493e-02,   5.12034901e-02],
        ..., 
        [  3.45224204e+01,  -1.75614678e+01,  -1.94484609e+01],
        [  3.29415506e+01,  -2.51417066e+01,  -1.38806680e+01],
        [  3.73191277e+01,  -1.27082499e+01,  -1.92797551e+01]]),
 '

In [38]:
import pandas as pd

stars_df = pd.DataFrame()

stars_df['M'] = stars['Masses']
stars_df['x'] = stars['Coordinates'][:,0]
stars_df['y'] = stars['Coordinates'][:,1]
stars_df['z'] = stars['Coordinates'][:,2]
stars_df['Vx'] = stars['Velocities'][:,0]
stars_df['Vy'] = stars['Velocities'][:,1]
stars_df['Vz'] = stars['Velocities'][:,2]
stars_df['Metallicity'] = stars['GFM_Metallicity']
stars_df['U'] = stars['Potential']
stars_df['StellarFormationTime'] = stars['GFM_StellarFormationTime']
#stars_df['Distance_center'] = stars['Distance_to_center']
stars_df['Wx'] = stars['Angular_Momentum'][:,0]
stars_df['Wy'] = stars['Angular_Momentum'][:,1]
stars_df['Wz'] = stars['Angular_Momentum'][:,2]
stars_df['Specific_energy'] = stars['Specific_Energy']
stars_df['Circularity'] = stars['Circularity']

stars_df.to_csv('stars_5.0rHalf_nometalcutoff_dataframe.csv')

In [ ]:
np.savetxt('datos_2.0rHalf_1.0metalcut_circularidades.csv',stars['Circularity'],delimiter=',')

In [4]:
## Utilizando fu nciones creadas para la circularidad
subhaloID = 394621
snapNum = 99
basepath = 'sims.TNG/TNG50-1/output'
radius_fraction = 5.0
metallicity_cutoff = False


stars = circularities_diagmethod(subhaloID,snapNum,basepath,radius_fraction,metallicity_cutoff)
stars

8.78085600375
Autovalores 
 [  5.25069582e+11+0.j   5.30789761e+11+0.j   9.86400227e+11+0.j] 
 Autovectores 
 [[-0.61993992 -0.38497245  0.68372154]
 [ 0.32870445  0.66380388  0.6717962 ]
 [ 0.71248007 -0.64121807  0.28498179]]
[[  5.25288964e+11   5.46697088e+08   9.99753523e+09]
 [  5.46606592e+08   5.31649364e+11   1.97927813e+10]
 [  9.99541043e+09   1.97921014e+10   9.85320718e+11]] |<-----Tensor de inercia Diagonalizado
-----------------------
 [-79299828.19766761 -76400854.07738566  92124138.48795892]


{'count': 1317404,
 'Masses': array([ 46668.125     ,  44963.171875  ,  49238.66796875, ...,
         61256.86328125,  88369.1171875 ,  46834.1640625 ], dtype=float32),
 'Coordinates': array([[ -3.93353986e-02,  -1.31864323e-02,  -5.99267291e-03],
        [ -6.49641909e-02,  -3.04149412e-02,   1.87027536e-02],
        [ -2.27459847e-02,  -5.49636265e-02,  -3.43350988e-03],
        ..., 
        [  1.71789163e+01,   3.86879455e+00,  -3.96025717e+01],
        [  2.44241809e+01,  -1.32723269e+00,  -3.62163382e+01],
        [  1.21966184e+01,   3.08081298e+00,  -4.20437435e+01]]),
 'Velocities': array([[  2.53646179e-02,  -2.57088907e-02,  -2.12781139e-02],
        [  6.47700448e-02,  -2.89792055e-02,  -2.14533158e-02],
        [  3.29127517e-02,  -4.62687281e-02,   1.80590045e-02],
        ..., 
        [ -3.92162558e+01,  -1.83901519e+01,  -1.52689428e+00],
        [ -3.93923571e+01,  -1.71828958e+01,   7.93198336e+00],
        [ -3.74932580e+01,  -2.21909090e+01,  -5.26709277e+00]]),
 '

# representando graficamente


In [19]:
import matplotlib.pyplot as plt
import matplotlib as mpl

index = np.where( (stars['Circularity'] >= -1) & (stars['Circularity'] <= 1.5) )

plt.hist(stars['Circularity'][index],bins=800)
plt.show()

<Figure size 640x480 with 1 Axes>

In [44]:
print('test')

test
